In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Content of Table

# Import & transform data

## Import data

In [2]:
DATA_DIR = Path('data/single_request')
LATENCY_CSV = 'single_request_latency.csv'
SERVER_USAGE_CSV = 'single_request_server_usage.csv'
CLIENT_USAGE_CSV = 'single_request_client_usage.csv'

In [3]:
df_latency = pd.read_csv(DATA_DIR / LATENCY_CSV)
df_server_usage = pd.read_csv(DATA_DIR / SERVER_USAGE_CSV)
df_client_usage = pd.read_csv(DATA_DIR / CLIENT_USAGE_CSV)

## Transform data

### Standardise timestamps

Offset timestamps to a common baseline, which is the server start time for each epoch

In [4]:
def offset_timestamp_to_base(df, ts_cols: str | list[str], base_col='perf_base_ns'):
    """
    Offsets ts_col by subtracting base_col so timestamps start at zero.
    """
    cols = [ts_cols] if isinstance(ts_cols, str) else ts_cols

    for col in cols:
        df[col] = df[col] - df[base_col]

offset_timestamp_to_base(df_latency, ['t0', 't_req', 't_res', 't_in', 't_out'])
offset_timestamp_to_base(df_server_usage, 'ts')
offset_timestamp_to_base(df_client_usage, 'ts')

### Calculate Duration

Caluclate duration for each specific phase

In [5]:
# 1. Client “setup” only: create req, headers/URL, channel, in‐memory object
df_latency['client_setup_ns'] = df_latency['t_req'] - df_latency['t0']

# 2. Uplink latency (client→server network + server receive/parse)
df_latency['uplink_latency_ns'] = df_latency['t_in']  - df_latency['t_req']

# 3. Pure server processing (handler logic + response serialisation)
df_latency['server_processing_ns'] = df_latency['t_out'] - df_latency['t_in']

# 4. Downlink latency (server→client network + client parse/deserialisation)
df_latency['downlink_latency_ns'] = df_latency['t_res'] - df_latency['t_out']

Calculate end-to-end duration

In [6]:
# 1. Client‐observed round‐trip (serialisation + network + server + deserialisation)
df_latency['round_trip_ns'] = df_latency['t_res'] - df_latency['t_req']

# 2. Total in‐app runtime (setup + round‐trip)
df_latency['total_runtime_ns'] = df_latency['t_res'] - df_latency['t0']

In [7]:
# 1. Client‐observed round‐trip (serialisation + network + server + deserialisation)
df_latency['round_trip_ns'] = df_latency['size'] / df_latency['round_trip_ns']

# 2. Total in‐app runtime (setup + round‐trip)
df_latency['total_runtime_ns'] = df_latency['t_res'] - df_latency['t0']

### Caulculate speed

In [8]:
df_latency['items_per_ms_round_trip'] = (df_latency['size'] / df_latency['round_trip_ns']) * 1_000_000
df_latency['items_per_ms_round_trip'] = (df_latency['size'] / df_latency['total_runtime_ns']) * 1_000_000

## Inspection

In [9]:
df_latency.head()

,mode,size,req_id,t0,t_req,t_res,t_in,t_out,perf_base_ns,epoch_base_ns,req_size_bytes,res_size_bytes,client_setup_ns,uplink_latency_ns,server_processing_ns,downlink_latency_ns,round_trip_ns,total_runtime_ns,items_per_ms_round_trip
0,grpc,1,d6fc5c563cada9bc,343614125,347422000,352247833,351662125,351862083,37801799555250,1747488516924895000,2,100,3807875,4240125,199958,385750,2.072181e-07,8633708,0.115825
1,grpc,1,fdd5cc376181e068,482784791,485150916,487121333,486641250,486794041,37801799555250,1747488516924895000,2,100,2366125,1490334,152791,327292,5.075068e-07,4336542,0.230598
2,grpc,1,5ab938f342d65c35,616468000,618739000,620799833,620324875,620460500,37801799555250,1747488516924895000,2,100,2271000,1585875,135625,339333,4.852407e-07,4331833,0.230849
3,grpc,1,b80220b2b1266432,752450708,754810125,756896458,756282041,756555000,37801799555250,1747488516924895000,2,100,2359417,1471916,272959,341458,4.793099e-07,4445750,0.224934
4,grpc,1,7c30bc491bcbff11,887355208,889608000,891538250,891064625,891176125,37801799555250,1747488516924895000,2,100,2252792,1456625,111500,362125,5.180676e-07,4183042,0.239060


In [10]:
df_server_usage.head()

,protocol,size,usage_side,ts,rss,cpu,perf_base_ns,epoch_base_ns
0,grpc,1,server,67553458,16015360,0.5,37801799555250,1747488516924895000
1,grpc,1,server,69210583,16154624,0.2,37801799555250,1747488516924895000
2,grpc,1,server,70512000,16261120,0.2,37801799555250,1747488516924895000
3,grpc,1,server,71805083,16384000,0.2,37801799555250,1747488516924895000
4,grpc,1,server,73092791,16515072,0.3,37801799555250,1747488516924895000


In [11]:
df_client_usage.head()

,protocol,size,usage_side,ts,rss,cpu,perf_base_ns,epoch_base_ns
0,grpc,1,client,13694464458,19202048,0.7,37801799555250,1747488516924895000
1,grpc,1,client,13696110416,20123648,2.5,37801799555250,1747488516924895000
2,grpc,1,client,13697413041,20365312,2.4,37801799555250,1747488516924895000
3,grpc,1,client,13698707375,20488192,1.5,37801799555250,1747488516924895000
4,grpc,1,client,13700001583,20946944,2.4,37801799555250,1747488516924895000


# Analysis

## Descriptive statistics

In [12]:
df_latency.columns

Index(['mode', 'size', 'req_id', 't0', 't_req', 't_res', 't_in', 't_out',
       'perf_base_ns', 'epoch_base_ns', 'req_size_bytes', 'res_size_bytes',
       'client_setup_ns', 'uplink_latency_ns', 'server_processing_ns',
       'downlink_latency_ns', 'round_trip_ns', 'total_runtime_ns',
       'items_per_ms_round_trip'],
      dtype='object')

df_lantency